In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-03-05 07:27:16,667 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 07:27:16,670 - settings - Setting database.user to celiib
INFO - 2021-03-05 07:27:16,670 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 07:27:16,674 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-05 07:27:16,675 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 07:27:16,688 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-05 07:27:17,008 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 07:27:17,012 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 07:27:17,012 - settings - Setting database.user to celiib
INFO - 2021-03-05 07:27:17,013 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 07:27:17,014 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 07:27:17,015 - settings - Setting database.user to celiib
INFO - 2021-03-05 07:27:17,015 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 07:27:17,016 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-05 07:27:17,380 - settings - Setting enable_python_native_blobs to True


In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = True

# Debugging the contains method

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-05 07:27:17,489 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 07:27:17,489 - settings - Setting database.user to celiib
INFO - 2021-03-05 07:27:17,490 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 07:27:17,492 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-05 07:27:17,493 - settings - Setting database.user to celiib
INFO - 2021-03-05 07:27:17,493 - settings - Setting database.password to newceliipass
INFO - 2021-03-05 07:27:17,494 - settings - Setting enable_python_native_blobs to True


Sleeping 31 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-03-05 07:27:17,912 - settings - Setting enable_python_native_blobs to True


In [5]:
#(schema.jobs & "table_name='__decomposition'").delete()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 5960


In [8]:
decimation_version = 0
decimation_ratio = 0.25

key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())
key_source                 

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988608634,0,0.25
864691135065024068,0,0.25
864691135155711844,0,0.25
864691135210232512,0,0.25
864691135341057989,0,0.25
864691135345291423,0,0.25
864691135373423816,0,0.25
864691135373676873,0,0.25
864691135382990571,0,0.25
864691135463533125,0,0.25


In [9]:
#minnie.DecompositionVersions.insert1(dict(process_version=6,description="3/4: retry limb processing with meshparty if meshafterparty fails"),skip_duplicates=True)

In [10]:
# schema.external['decomposition'].delete(delete_external_files=True)
# schema.external['somas'].delete(delete_external_files=True)
# schema.external['faces'].delete(delete_external_files=True)
# # schema.external['decimated_meshes'].delete(delete_external_files=True)

In [11]:
# key_source = ((minnie.Decimation).proj(decimation_version='version') & 
#                             "decimation_version=" + str(decimation_version) &
#                        f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())               
# minnie.Decomposition() & key_source

In [12]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 6

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())               
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)
        
        keys_to_delete = ["axon_length",
        "axon_area",
        "max_soma_volume",
        "max_soma_n_faces"]
        
        for k_to_delete in keys_to_delete:
            del new_key[k_to_delete]

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [13]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,0e988b84634cec4b9077322ad1512aac,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.131,at-node4,19,203874,2021-03-05 01:06:37
__decomposition,14d2292cca778882cd1343a58676530f,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,19,203866,2021-03-05 01:12:24
__decomposition,18fad79fc6da95c8e54588d9e7db4027,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.133,at-node6,19,203883,2021-03-05 01:14:38
__decomposition,223292d754d01c10ec97cae17b086998,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.133,at-node6,19,203882,2021-03-05 01:06:57
__decomposition,24c854aeea9a7afd26c48327ed452981,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,19,203862,2021-03-05 01:13:39
__decomposition,2e0af41fae70ea67ae01dbf37a82ebdf,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.155,at-node28,19,203877,2021-03-05 01:17:50
__decomposition,3756476d71c1ec85d9ce18d0940e8827,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.175,at-node48,19,203872,2021-03-05 01:12:16
__decomposition,3e910899c185a47a485f29c7b99746f1,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.155,at-node28,21,203879,2021-03-05 01:10:11
__decomposition,421d96cd38cb57efc18a96b2210cdbef,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,19,203869,2021-03-05 01:13:15
__decomposition,53a4c468ff5743e1e28cc19c0c217247,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.147,at-node20,19,203880,2021-03-05 01:08:21


In [14]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-03-05 07:27:23,157 - autopopulate - Found 21 keys to populate
INFO - 2021-03-05 07:27:23,182 - connection - Transaction started
INFO - 2021-03-05 07:27:23,184 - autopopulate - Populating: {'segment_id': 864691136672813959, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691136672813959 ----
somas = [[<trimesh.Trimesh(vertices.shape=(21521, 3), faces.shape=(42276, 3))>, <trimesh.Trimesh(vertices.shape=(15596, 3), faces.shape=(31018, 3))>], array([597.7325, 597.7325]), array([0.766, 0.882])]

--- Beginning preprocessing of 864691136672813959---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(21521, 3), faces.shape=(42276, 3))>, <trimesh.Trimesh(vertices.shape=(15596, 3), faces.shape=(31018, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(21521, 3), faces.shape=(42276, 3))>, <trimesh.Trimesh(vertices.shape=(15596, 3), faces.shape=(31018, 3))>]
soma_mesh_list_centers = [array([589754.7233214 , 837130.07899262, 863307.21379583]), array([716900.1115286 , 972726.06945371, 924998.0104514 ])]
Getting Glia and Nuclei Pieces S

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/388547 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:13<00:00, 13.26s/it]


...

  0%|          | 0/18 [00:00<?, ?it/s]

AFTER face_lookup_resolved_test


  0%|          | 0/2 [00:00<?, ?it/s]

main_branch = 61
Total time for stitching floating pieces = 9.784603357315063


Exception: The number of starting nodes found was not exactly one: []